先导入一些必要模块

In [1]:
import numpy as np
import warp as wp

def warp_coo_deduplicate(rows, cols, vals):
    """
    去重 COO 格式，vals 为 3x3 矩阵块，只做 sum 聚合
    """
    rows_np = rows.numpy()
    cols_np = cols.numpy()
    vals_np = vals.numpy()  # shape (nnz, 3, 3)
    
    # --- 1. 构造唯一索引 ---
    max_col = np.max(cols_np) if len(cols_np) > 0 else 0
    idx = rows_np * (max_col + 1) + cols_np  # 把 (row, col) 映射为唯一整数

    # --- 2. 获取去重信息 ---
    unique_idx, inv = np.unique(idx, return_inverse=True)
    n_unique = len(unique_idx)

    # --- 3. 反推去重后的行列索引 ---
    out_rows_np = unique_idx // (max_col + 1)
    out_cols_np = unique_idx % (max_col + 1)

    # --- 4. 对应位置累加 3x3 块 ---
    vals_flat = vals_np.reshape(vals_np.shape[0], -1)   # (nnz, 9)
    out_vals_flat = np.zeros((n_unique, 9), dtype=vals_np.dtype)
    np.add.at(out_vals_flat, inv, vals_flat)  # 用 inv 实现聚合加法
    out_vals_np = out_vals_flat.reshape(n_unique, 3, 3)

    # --- 5. 如果只有一个元素且为全零，则直接返回 False ---
    if len(out_rows_np) == 1:
        if np.allclose(out_vals_np[0], 0.0):
            return (False, False, False)

    # --- 6. 返回 Warp 数组 ---
    return (
        wp.array(out_rows_np, dtype=int),
        wp.array(out_cols_np, dtype=int),
        wp.array(out_vals_np, dtype=vals.dtype)
    )





In [2]:
# ------------------- 测试 -------------------
if __name__ == "__main__":
    # 构造一个重复的 COO 示例
    rows = wp.array([0, 0, 1, 1, 1], dtype=int)
    cols = wp.array([0, 0, 1, 1, 2], dtype=int)

    # 每个块是 3x3 矩阵，这里随便造点值
    vals = wp.array([
        np.eye(3) * 1.0,   # 对角矩阵 1
        np.eye(3) * 2.0,   # 重复 (0,0)
        np.eye(3) * 3.0,   # (1,1)
        np.eye(3) * 4.0,   # 重复 (1,1)
        np.eye(3) * 5.0,   # (1,2)
    ], dtype=float)

    out_rows, out_cols, out_vals = warp_coo_deduplicate(rows, cols, vals)

    print("去重后行号:", out_rows)
    print("去重后列号:", out_cols)
    print("去重后块值:\n", out_vals)

Warp 1.10.0.dev20250927 initialized:
   Git commit: 1049c9e9a6c5d8a5c001cfa93e1ebe9eabe96e5b
   CUDA driver not found or failed to initialize
   Devices:
     "cpu"      : "Intel64 Family 6 Model 186 Stepping 2, GenuineIntel"
   Kernel cache:
     \\?\C:\Users\baiix\AppData\Local\NVIDIA\warp\Cache\1.10.0.dev20250927
去重后行号: [0 1 1]
去重后列号: [0 1 2]
去重后块值:
 [[[3. 0. 0.]
  [0. 3. 0.]
  [0. 0. 3.]]

 [[7. 0. 0.]
  [0. 7. 0.]
  [0. 0. 7.]]

 [[5. 0. 0.]
  [0. 5. 0.]
  [0. 0. 5.]]]


In [3]:
import numpy as np
import warp as wp

def remove_fixed_blocks(rows_np, cols_np, vals_np, flag_all_particle):
    """
    从 COO (rows, cols, vals) 中删除涉及 fixed_points 的块，
    并根据 flag_all_particle 进行行列偏移。
    """
    rows_np = np.asarray(rows_np)
    cols_np = np.asarray(cols_np)
    vals_np = np.asarray(vals_np)
    flag_all_particle = np.asarray(flag_all_particle)

    nnz = len(rows_np)
    keep_mask = np.ones(nnz, dtype=bool)

    # 1️⃣ 找出涉及固定点的条目
    for i in range(nnz):
        if flag_all_particle[rows_np[i]] == -1 or flag_all_particle[cols_np[i]] == -1:
            keep_mask[i] = False

    # 2️⃣ 保留非固定条目
    rows_np = rows_np[keep_mask]
    cols_np = cols_np[keep_mask]
    vals_np = vals_np[keep_mask]

    # 3️⃣ 应用偏移：新索引 = 原索引 - flag_all_particle[原索引]
    rows_np = rows_np - flag_all_particle[rows_np]
    cols_np = cols_np - flag_all_particle[cols_np]

    return (
        wp.array(rows_np, dtype=int),
        wp.array(cols_np, dtype=int),
        wp.array(vals_np, dtype=wp.mat33)
    )





In [4]:
# ---------------------- 测试 ----------------------
if __name__ == "__main__":
    # 原始 5 个点中，第 1 和第 3 个点固定（索引 0-based：1, 3）
    # flag_all_particle[i]:
    #   -1 代表固定点
    #   >=0 表示删除固定点后的偏移量
    flag_all_particle = np.array([0, -1, 1, -1, 2], dtype=int)
    # 解释：
    #   原索引 [0,1,2,3,4]
    #   删除固定点 1,3 后：
    #   新索引 [0, -, 1, -, 2]
    #   因此非固定点的偏移 = 已删除的固定点数量：
    #     0→0, 1→-1, 2→1, 3→-1, 4→2

    # 模拟原始 COO 格式索引 (包含固定点)
    rows_np = np.array([0, 1, 2, 3, 4, 2, 4])
    cols_np = np.array([0, 1, 2, 3, 4, 4, 1])
    vals_np = np.array([np.eye(3) * i for i in range(len(rows_np))])

    print("原始 rows:", rows_np)
    print("原始 cols:", cols_np)
    print("固定点标志 flag_all_particle:", flag_all_particle)

    out_rows, out_cols, out_vals = remove_fixed_blocks(rows_np, cols_np, vals_np, flag_all_particle)

    print("\n过滤后 rows:", out_rows.numpy())
    print("过滤后 cols:", out_cols.numpy())
    print("过滤后 vals 对角元素:", [np.diag(v) for v in out_vals.numpy()])

原始 rows: [0 1 2 3 4 2 4]
原始 cols: [0 1 2 3 4 4 1]
固定点标志 flag_all_particle: [ 0 -1  1 -1  2]

过滤后 rows: [0 1 2 1]
过滤后 cols: [0 1 2 2]
过滤后 vals 对角元素: [array([0., 0., 0.], dtype=float32), array([2., 2., 2.], dtype=float32), array([4., 4., 4.], dtype=float32), array([5., 5., 5.], dtype=float32)]


In [6]:
if None:
    print('None')

来一个拼接数组

In [1]:
import numpy as np

# 创建两个数组
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6], [7, 8]])

# 方法1：沿行（axis=0）拼接，相当于垂直堆叠
c1 = np.concatenate((a, b), axis=0)
print("沿行拼接:\n", c1)

# 方法2：沿列（axis=1）拼接，相当于水平堆叠
c2 = np.concatenate((a, b), axis=1)
print("沿列拼接:\n", c2)

# 方法3：vstack（垂直堆叠）
c3 = np.vstack((a, b))
print("vstack:\n", c3)

# 方法4：hstack（水平堆叠）
c4 = np.hstack((a, b))
print("hstack:\n", c4)

# 方法5：stack（沿新轴堆叠）
c5 = np.stack((a, b), axis=0)
print("stack沿新轴:\n", c5)


沿行拼接:
 [[1 2]
 [3 4]
 [5 6]
 [7 8]]
沿列拼接:
 [[1 2 5 6]
 [3 4 7 8]]
vstack:
 [[1 2]
 [3 4]
 [5 6]
 [7 8]]
hstack:
 [[1 2 5 6]
 [3 4 7 8]]
stack沿新轴:
 [[[1 2]
  [3 4]]

 [[5 6]
  [7 8]]]
